In [2]:
import os
import numpy as np
import plotly.graph_objects as go
from Bio import AlignIO, SeqIO
import subprocess

In [2]:
def extract_id(header):
    id_part = header.split('|')[1].split('-')[0]
    return id_part

def write_sequence_to_file(id_part, sequence, output_dir):
    filename = os.path.join(output_dir, f"{id_part}.fasta")
    
    with open(filename, 'a') as f:
        f.write(sequence)

def split_fasta_by_id():
    fasta_file = 'data/uniprot_analysis/uniprotkb_reviewed_true_AND_model_organ_2024_09_11.fasta'
    output_dir = 'data/uniprot_analysis/output-id'    
    current_sequence = ""
    current_id = ""
    
    with open(fasta_file, 'r') as f:
        for line in f:
            if line.startswith('>'):
                if current_sequence and current_id:
                    write_sequence_to_file(current_id, current_sequence, output_dir)
                
                current_sequence = line
                current_id = extract_id(line)
            else:
                current_sequence += line
        
        if current_sequence and current_id:
            write_sequence_to_file(current_id, current_sequence, output_dir)

In [2]:

fasta_file = '/home/talnawa/Desktop/protein_sequencing/data/uniprot_analysis/uniprotkb_reviewed_true_AND_model_organ_2024_09_11.fasta'
count = 0
with open(fasta_file, 'r') as f:
    for line in f:
        if line.startswith('>'):
            count += 1
print (count)

42523


In [3]:
def count_sequences_in_file(fasta_file):
    sequence_count = 0
    with open(fasta_file, 'r') as f:
        for line in f:
            if line.startswith('>'): 
                sequence_count += 1
    #if sequence_count > 15:
    #    print(f"{fasta_file} has {sequence_count} sequences.")
    return sequence_count

def create_histogram_data(fasta_dir):
    sequence_counts = []

    for fasta_file in os.listdir(fasta_dir):
        file_path = os.path.join(fasta_dir, fasta_file)
        count = count_sequences_in_file(file_path)
        sequence_counts.append(count)

    return sequence_counts

def plot_histogram():
    fasta_dir = '/home/talnawa/Desktop/protein_sequencing/data/uniprot_analysis/output-id'
    sequence_counts = create_histogram_data(fasta_dir)
    
    fig = go.Figure(data=[go.Histogram(x=sequence_counts, nbinsx=max(sequence_counts))])
    perc_90 = np.percentile(sequence_counts, 90)
    perc_95 = np.percentile(sequence_counts, 95)
    perc_99 = np.percentile(sequence_counts, 99)

    fig.add_vline(x=perc_90+0.5, line=dict(color='blue', dash='dash'), annotation_text='90%', annotation_position='top left')
    fig.add_vline(x=perc_95+0.5, line=dict(color='green', dash='dash'), annotation_text='95%', annotation_position='top left')
    fig.add_vline(x=perc_99+0.5, line=dict(color='red', dash='dash'), annotation_text='99%', annotation_position='top left')


    fig.update_layout(
        title="Histogram of Isoforms per Protein",
        xaxis_title="Number of Isoforms per Protein",
        yaxis_title="Number of Proteins",
        bargap=0.2
    )
    fig.show()

plot_histogram()

In [ ]:
def get_alignment(input_file: str | os.PathLike):
    records = list(SeqIO.parse(input_file, 'fasta'))

    # write to temporary file and do alignment
    padded_sequences_path = f'/home/talnawa/Desktop/protein_sequencing/data/tmp/{os.path.splitext(os.path.basename(input_file))[0]}_padded'
    with open(f"{padded_sequences_path}.fasta", 'w') as f:
        SeqIO.write(records, f, 'fasta')

    # penalties für gap öffnung
    cmd = f"./clustalw-2.1-linux/clustalw2 -infile={padded_sequences_path}.fasta -GAPOPEN=10 -GAPEXT=0.2"
    subprocess.run(cmd, shell=True, stdout=subprocess.PIPE, text=True)
    
    align = AlignIO.read(f"{padded_sequences_path}.aln", "clustal")
    
    os.remove(f"{padded_sequences_path}.fasta")
    return align

def align_and_save_sequences():
    input_dir = '/home/talnawa/Desktop/protein_sequencing/data/uniprot_analysis/output-id'
    output_dir = '/home/talnawa/Desktop/protein_sequencing/data/uniprot_analysis/output_aligned_2'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for fasta_file in os.listdir(input_dir):
        input_file = os.path.join(input_dir, fasta_file)
        if count_sequences_in_file(input_file) > 1:
            sequence_id = os.path.splitext(fasta_file)[0]

            # Perform alignment
            alignment = get_alignment(input_file)

            # Write the aligned sequences to the output directory
            output_file = os.path.join(output_dir, f"{sequence_id}_aligned.fasta")
            with open(output_file, 'w') as f:
                AlignIO.write(alignment, f, 'fasta')

align_and_save_sequences()

In [11]:
# Check alignment
def check_alignment():
    aligned_dir = '/home/talnawa/Desktop/protein_sequencing/data/uniprot_analysis/output_aligned_2'
    result = {}
    for aligned_file in os.listdir(aligned_dir):
        aligned_file_path = os.path.join(aligned_dir, aligned_file)
        alignments = AlignIO.read(aligned_file_path, 'fasta')
        max_sequence_length = 0
        for alignment in alignments:
            if len(alignment.seq) > max_sequence_length:
                max_sequence_length = len(alignment.seq)

        assert all(len(alignment.seq) == max_sequence_length for alignment in alignments)

        different_possibilities = [-1]*max_sequence_length
        for i in range(len(alignments[0].seq)):
            proteins = set()
            for alignment in alignments:
                protein = alignment.seq[i]
                proteins.add(protein)
            
            if '-' in proteins:
                if len(proteins) == 2:
                    different_possibilities[i] = -1
                if len(proteins) > 2:
                    different_possibilities[i] = len(proteins)-1
            else:
                different_possibilities[i] = len(proteins)
        if max(different_possibilities) in result:
            result[max(different_possibilities)] += 1
        else:
            result[max(different_possibilities)] = 1
    return result
different_possibilities_per_alignment = check_alignment()

print(different_possibilities_per_alignment)

KeyboardInterrupt: 

In [9]:
different_possibilities_per_alignment[1] = 3228
del different_possibilities_per_alignment[-1]

KeyError: -1

In [13]:
print(different_possibilities_per_alignment)

{2: 6414, 3: 768, 1: 3228, 6: 10, 4: 125, 5: 28, 7: 1, 8: 2, 10: 1}


In [12]:
# Plot the results as histogram
def plot_alignment_results():
    fig = go.Figure(data=[go.Bar(x=list(different_possibilities_per_alignment.keys()), y=list(different_possibilities_per_alignment.values()))])
    fig.update_layout(
        title="Number of Different Amino Acids per Position in the Alignment",
        xaxis_title="Number of Different Amino Acids",
        yaxis_title="Number of Alignments"
    )
    fig.show()

plot_alignment_results()

In [14]:

fasta_dir = '/home/talnawa/Desktop/protein_sequencing/data/uniprot_analysis/output-id'
print(len(os.listdir(fasta_dir)))
print(f"Percentage of Gens with 0 or 1 different exons: {13191/20435*100}%")
print(f"Percentage of Gens with 2 different exons: {6414/20435*100}%")
print(f"Percentage of Gens with 3 or more different exons: {935/20435*100}%")

20435
Percentage of Gens with 0 or 1 different exons: 64.55101541472963%
Percentage of Gens with 2 different exons: 31.38732566674823%
Percentage of Gens with 3 or more different exons: 4.575483239540005%
